In [17]:
import random
import os
import shutil
import glob
import gc
import sys
import h5py
import time
import datetime
import pickle
import librosa
import warnings
import matplotlib.pyplot as plt

import numpy as np
from numpy import array
import pandas as pd
from pandas.plotting import parallel_coordinates
from pydub import AudioSegment

#imported for testing
import wave
import contextlib

# for outputing file
from scipy.cluster.vq import vq, kmeans, whiten
import scipy.stats.stats as st

from sklearn.cluster import KMeans
from sklearn.datasets import make_classification
from sklearn.datasets.samples_generator import make_blobs
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize
from sklearn.metrics import auc, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.utils import class_weight

import keras

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.layers import Convolution1D, Dense, MaxPooling1D, Flatten, Add, Dropout, Input, Activation
from keras.layers import TimeDistributed, Bidirectional, LSTM, LeakyReLU
from keras.models import Sequential
from keras import optimizers, regularizers
from keras.utils import np_utils, to_categorical

#from colorama import Fore, Back, Style

from IPython.display import clear_output


#warnings.filterwarnings('ignore')
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
# assert 'GPU' in str(device_lib.list_local_devices())

# confirm Keras sees the GPU
from keras import backend
# print(len(backend.tensorflow_backend._get_available_gpus()) > 0)

#warnings.filterwarnings('ignore')
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

sys.path.insert(1, '..//components//')
import load_feat_directories

In [18]:
def empirical_class_mean(X, Y, c, func, penultimate_neuron_num):
    """
    X is of shame (batch, num_of_small_segs, feats_in_one_seg).
    Y is of shape (batch, class).
    c is a class, of type integer.
    func is the output of the penultimate layer of DNNs. 
    func should be called as func(x), s.t. x is one xample in X.
    """
    N_c = 0
    for item in Y:
        if np.amax(item) == c:
            N_c += 1
    sum_func = np.array([0] * penultimate_neuron_num)
    
    for index in range(0, len(Y)):
        if Y[index] == c:
            sum_func = np.add(func(X[index]), sum_func)
    return sum_func / N_c


def empirical_class_means(X, Y, C, func, penultimate_neuron_num):
    """
    Calculate the empirical class means for all classes and output a numpy array.
    """
    result = []
    for c in C:
        miu = empirical_class_mean(X, Y, c, func, penultimate_neuron_num)
        result.append(miu)
    return np.array(result)

def emprical_covariance(X, Y, C, func, penultimate_neuron_num):
    assert len(X) == len(Y)
    N = len(Y)
    
    mius = empirical_class_means(X, Y, C, func, penultimate_neuron_num)
    miu = mius[0]
    
    sum_all_classes = np.array(miu.shape)
    
    print(mius)
    
    for c in C:
        sum_single_c = np.array(miu.shape)
        for index in range(0, len(Y)):
            if Y[index] == c:
                miu = mius[c]
                difference = func(X[index]) - miu
                transpose = np.transpose(difference)
                result = difference * transpose
                sum_single_c = np.add(sum_single_c, result)
        
        sum_all_classes = np.add(sum_all_classes, sum_single_c)
        
    covariance = sum_all_classes/N
    
    return covariance

In [19]:
sample_rate = 44100
frame_number = 48
hop_length = 441  # frame size= 2 * hop
segment_length = int(sample_rate * 0.2)  # 0.2
segment_pad = int(sample_rate * 0.02)     # 0.02
overlapping = int(sample_rate * 0.1)   # 0.1

classes = 5
NumofFeaturetoUse = 272
n_neurons = 1024 * 4
dense_layers = 10
num_layers = 3
fillength = 3
nbindex = 1024 * 3
dropout = 0.2
n_batch = 128
n_epoch = 1000

In [20]:
all_noised_npy = load_feat_directories.allnoised_npy
all_noised_npy_test = load_feat_directories.allnoised_npy_test
home_noised_npy = load_feat_directories.homenoised_npy
home_noised_npy_test = load_feat_directories.homenoised_npy_test

for index in range(0, 5):
    if not os.path.exists(all_noised_npy[index]):
        print(all_noised_npy[index] + ' does not exist. Breaking the loop... ')
        break

    if not os.path.exists(home_noised_npy[index]):
        print(home_noised_npy[index] + 'does not exist. Breaking the loop... ')
        break

In [21]:
def comprise_vector(path):
    vec_to_return = []
    for fname in os.listdir(path):
        current_vec = np.load(path + fname)
        vec_to_return.append(current_vec)

    vec_to_return = np.array(vec_to_return)
    return vec_to_return

def comprise_label(feature_vector, label):
    label_vec_to_ret = []
    length = len(list(feature_vector))
    for index in range(0, length):
        current_label = [label]
        label_vec_to_ret.append(current_label)
    label_vec_to_ret = np.array(label_vec_to_ret)

    return label_vec_to_ret

def float_compatible(input_np):
    x = np.where(input_np >= np.finfo(np.float32).max)
    for index in range(0, len(x[0])):
        x_position = x[0][index]
        y_position = x[1][index]
        input_np[x_position, y_position] = 0.0
    input_np = np.nan_to_num(input_np)

    return input_np

In [22]:
for index in [0, 1, 2, 3, 4]:
    if not os.path.exists(home_noised_npy[index]):
        print(home_noised_npy[index] + 'does not exist.')
    else:
        path = home_noised_npy[index]
        print(path)
        if index == 0:
            h_feature_vector_home = comprise_vector(path)
            h_label_vector_home = comprise_label(h_feature_vector_home, index)
        elif index == 1:
            a_feature_vector_home = comprise_vector(path)
            a_label_vector_home = comprise_label(a_feature_vector_home, index)
        elif index == 2:
            n_feature_vector_home = comprise_vector(path)
            n_label_vector_home = comprise_label(n_feature_vector_home, index)
        elif index == 3:
            s_feature_vector_home = comprise_vector(path)
            s_label_vector_home = comprise_label(s_feature_vector_home, index)
        else:
            o_feature_vector_home = comprise_vector(path)
            o_label_vector_home = comprise_label(o_feature_vector_home, index)

for index in [0, 1, 2, 3, 4]:
    
    if not os.path.exists(home_noised_npy_test[index]):
        print(home_noised_npy_test[index] + 'does not exist.')
    else:
        path = home_noised_npy_test[index]
        print(path)
        if index == 0:
            h_feature_vector_home_test = comprise_vector(path)
            h_label_vector_home_test = comprise_label(h_feature_vector_home_test, index)
        elif index == 1:
            a_feature_vector_home_test = comprise_vector(path)
            a_label_vector_home_test = comprise_label(a_feature_vector_home_test, index)
        elif index == 2:
            n_feature_vector_home_test = comprise_vector(path)
            n_label_vector_home_test = comprise_label(n_feature_vector_home_test, index)
        elif index == 3:
            s_feature_vector_home_test = comprise_vector(path)
            s_label_vector_home_test = comprise_label(s_feature_vector_home_test, index)
        else:
            o_feature_vector_home_test = comprise_vector(path)
            o_label_vector_home_test = comprise_label(o_feature_vector_home_test, index)


C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Happy_npy//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Angry_npy//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Neutral_npy//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Sad_npy//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Other_npy//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Happy_npy_test//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Angry_npy_test//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Neutral_npy_test//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Sad_npy_test//
C://Users//yg9ca//Documents//Datasets//padded_deamplified_homenoised_reverberate

In [23]:
# Load training npy files
featureSet = float_compatible(np.vstack((h_feature_vector_home, a_feature_vector_home, n_feature_vector_home, s_feature_vector_home, o_feature_vector_home)))
Label = (np.vstack((h_label_vector_home, a_label_vector_home, n_label_vector_home, s_label_vector_home, o_label_vector_home)))

Label[Label == 0] = 0
Label[Label == 1] = 1
Label[Label == 2] = 2
Label[Label == 3] = 3
Label[Label == 4] = 4

Label = to_categorical(Label)
#featureSet = np.split(featureSet, np.array([NumofFeaturetoUse]), axis = 2)[0]
print('training data: ' + str(featureSet.shape))
print('training label: ' + str(Label.shape))

# Load testing npy files
featureSet_val = float_compatible(np.vstack((h_feature_vector_home_test, a_feature_vector_home_test, n_feature_vector_home_test, s_feature_vector_home_test, o_feature_vector_home_test)))
Label_val = (np.vstack((h_label_vector_home_test, a_label_vector_home_test, n_label_vector_home_test, s_label_vector_home_test, o_label_vector_home_test)))

Label_val[Label_val == 0] = 0
Label_val[Label_val == 1] = 1
Label_val[Label_val == 2] = 2
Label_val[Label_val == 3] = 3
Label_val[Label_val == 4] = 4

Label_val = to_categorical(Label_val)
#featureSet_val = np.split(featureSet_val, np.array([NumofFeaturetoUse]), axis = 2)[0]
print('evaluation data: ' + str(featureSet_val.shape))
print('evaluation label: ' + str(Label_val.shape))

training data: (39916, 48, 272)
training label: (39916, 5)
evaluation data: (8843, 48, 272)
evaluation label: (8843, 5)


In [24]:
def train_cnn(prefix, source):
    save_to_path = prefix
    checkpoint_filepath = prefix + "Checkpoint_" + source + title + ".hdf5"
    final_filepath = prefix + "Final_" + source + title + ".hdf5"

    if not os.path.exists(save_to_path):
        os.mkdir(save_to_path)

    X, X_test, Y, Y_test= train_test_split(featureSet, Label, test_size = 0.25, shuffle = True)
    model = create_cnn(title, num_layers, n_neurons, n_batch, nbindex, dropout, classes, dense_layers)
    checkpoint = ModelCheckpoint(checkpoint_filepath, monitor = 'val_acc', verbose = 0, save_best_only = True, mode = 'auto')
    early_stopping_monitor = EarlyStopping(patience = 50)
    callbacks_list = [checkpoint, early_stopping_monitor]
    model.fit(X, Y, nb_epoch = n_epoch, batch_size = n_batch,  callbacks = callbacks_list, validation_data = (X_test, Y_test), verbose = 1)
    #model.save_weights(final_filepath)
    #model.load(checkpoint_filepath)
    
    return model

In [25]:
def max_index(a): 
    a = list(a)
    # inbuilt function to find the position of minimum  
    minpos = a.index(min(a))  
    # inbuilt function to find the position of maximum  
    maxpos = a.index(max(a))    
    return maxpos

def predict_cnn(model):
    y_pred = []
    y_true = []

    for item in list(Label_val):
            y_true.append(max_index(item))

    for item in list(model.predict(featureSet_val)):
            y_pred.append(max_index(item))

    print('Accuracy: ' + str(accuracy_score(y_true, y_pred)))
    #print('Precision: ' + str(precision_score(y_true, y_pred)))
    #print('Recall: ' + str(recall_score(y_true, y_pred)))
    #print('f1 score: ' + str(f1_score(y_true, y_pred)))

    #tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # Plot non-normalized confusion matrix
    #plot_confusion_matrix(y_true, y_pred, classes=[0, 1, 2, 3, 4],
    #                  title='Confusion matrix, without normalization')


In [26]:
adam = optimizers.Adam(lr = 1e-4, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0, amsgrad = True)
sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
rmsprop = optimizers.RMSprop(lr = 0.0001, rho = 0.9, epsilon = None, decay = 0.0)
adagrad = optimizers.Adagrad(lr = 0.01, epsilon = None, decay = 0.0)
adadelta = optimizers.Adadelta(lr = 1.0, rho = 0.95, epsilon = None, decay = 0.0)
adamax = optimizers.Adamax(lr = 0.002, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0)
nadam = optimizers.Nadam(lr = 0.002, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, schedule_decay = 0.004)

In [27]:
def record(str_message, log_file):
    str_message = str_message + '\n'
    file = open(log_file, 'a')
    file.write(str_message)
    file.close()

def create_cnn(title, num_layers, n_neurons, n_batch, nbindex, dropout, classes, dense_layers):

    model = Sequential()

    model.add(Convolution1D(nb_filter=nbindex, filter_length=fillength, activation = 'relu',
                            input_shape=(featureSet.shape[1], featureSet.shape[2]), kernel_constraint=maxnorm(3)))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(dropout))

    model.add(Convolution1D(nb_filter=nbindex*2, filter_length=fillength, activation = 'relu',
                            kernel_constraint=maxnorm(3)))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(dropout))

    model.add(Convolution1D(nb_filter=nbindex*2, filter_length=fillength, activation = 'relu',
                            kernel_constraint=maxnorm(3)))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(dropout))

    model.add(Convolution1D(nb_filter=nbindex, filter_length=fillength, activation = 'relu',
                            kernel_constraint=maxnorm(3)))  
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(dropout))
    
    model.add(Flatten())
    
    for i in range(0, dense_layers):
        model.add(Dense(n_neurons, activation = 'relu'))
        model.add(Dropout(dropout))
        
    # Here, get the empirical class mean and covariance of training samples

    model.add(Dense(classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=adam,
                  metrics=['accuracy'])

    return model

In [28]:
title = 'All_neurons_home_' + str(n_neurons) + '_filters_' + str(nbindex) + '_dropout_' + str(dropout) + '_epoch_' + str(n_epoch) + '_dense_' + str(dense_layers)

prefix = 'C://Users//yg9ca//Documents//modules//'
source = 'home_'

if not os.path.exists(prefix):
    os.mkdir(prefix)

final_filepath = prefix + "Checkpoint_" + source + title + ".hdf5"
model = create_cnn(title, num_layers, n_neurons, n_batch, nbindex, dropout, classes, dense_layers)

C:\Users\yg9ca\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", input_shape=(48, 272), kernel_constraint=<keras.con..., filters=3072, kernel_size=3)`
  if sys.path[0] == '':
C:\Users\yg9ca\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", kernel_constraint=<keras.con..., filters=6144, kernel_size=3)`
C:\Users\yg9ca\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", kernel_constraint=<keras.con..., filters=6144, kernel_size=3)`
C:\Users\yg9ca\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", kernel_constraint=<keras.con..., filters=3072, k

In [29]:
model = keras.models.load_model(prefix + "Checkpoint_" + source + title + ".hdf5")
predict_cnn(model)

Accuracy: 0.8792265068415696
